In [1]:
import pandas as pd
result_df = pd.read_csv("../../data/clean_data.500k.csv")
# result_df = pd.read_csv("../../data/clean_data.csv")
display(result_df)

,Unnamed: 0,event_dt,screen,feature,action,device_id,session_id,device_vendor,device_model,device_type,os,age,gender,is_finish,is_churn,node_id
0,0,2025-09-29 10:20:27+03:00,Еще,Открытие экрана,Переход к экрану,339,10000000009,Redmi,Redmi Note 12,phone,Android,70.0,Ж,0,0,875e50d28d26088a760866c1125ff124
1,1,2025-09-29 10:21:56+03:00,Еще,Открытие экрана,Переход к экрану,339,10000000009,Redmi,Redmi Note 12,phone,Android,70.0,Ж,0,0,875e50d28d26088a760866c1125ff124
2,2,2025-09-29 10:22:01+03:00,Еще,Переход в раздел 'Заявки',Тап на кнопку 'Заявки',339,10000000009,Redmi,Redmi Note 12,phone,Android,70.0,Ж,0,0,5281fb229131fa372bd15589fed81bc7
3,3,2025-09-29 10:23:49+03:00,Еще,Открытие экрана,Переход к экрану,339,10000000009,Redmi,Redmi Note 12,phone,Android,70.0,Ж,1,0,875e50d28d26088a760866c1125ff124
4,4,2025-09-29 10:20:17+03:00,Новая заявка,Выбор квартиры,Тап на квартиру,339,10000000009,Redmi,Redmi Note 12,phone,Android,70.0,Ж,0,0,e9c334108c17d0d6014b73f196b775f4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499994,499994,2025-09-29 16:08:38+03:00,Еще,Открытие экрана,Переход к экрану,44718,10000000425,Apple,iPhone 15 Pro Max,phone,iOS,48.0,М,1,0,875e50d28d26088a760866c1125ff124
499995,499995,2025-09-29 22:39:37+03:00,Еще,Открытие экрана,Переход к экрану,44718,10000000428,Apple,iPhone 15 Pro Max,phone,iOS,48.0,М,1,0,875e50d28d26088a760866c1125ff124
499996,499996,2025-09-29 23:08:27+03:00,Еще,Открытие экрана,Переход к экрану,44718,10000000431,Apple,iPhone 15 Pro Max,phone,iOS,48.0,М,1,0,875e50d28d26088a760866c1125ff124
499997,499997,2025-09-29 14:51:43+03:00,Еще,Открытие экрана,Переход к экрану,18497,10000001042,Apple,iPhone 11 Pro,phone,iOS,42.0,М,1,0,875e50d28d26088a760866c1125ff124


In [2]:
# Преобразование event_dt в datetime для корректной сортировки
result_df['event_dt'] = pd.to_datetime(result_df['event_dt'])

# Сортировка данных по устройству, сессии и времени
result_df = result_df.sort_values(['device_id', 'session_id', 'event_dt'])

# Создание столбца с следующей нодой в рамках одной сессии
result_df['next_node'] = result_df.groupby(['device_id', 'session_id'])['node_id'].shift(-1)

# Фильтрация: оставляем только строки, где есть следующая нода (переход)
transitions_df = result_df[['node_id', 'next_node']].dropna()
transitions_df.columns = ['source_id', 'target_id']

# Удаляем переходы из ноды в саму себя
transitions_df = transitions_df[transitions_df['source_id'] != transitions_df['target_id']]

# Оставляем только уникальные переходы
unique_transitions = transitions_df.drop_duplicates()

# Сохраняем в файл
unique_transitions.to_csv('../../data/links_graph.csv', index=False)

print(f"Таблица переходов сохранена в links.csv")
print(f"Количество уникальных переходов: {len(unique_transitions)}")

# Показать первые несколько строк для проверки
display(unique_transitions.head())


Таблица переходов сохранена в links.csv
Количество уникальных переходов: 1844


,source_id,target_id
219470,0072f89b60d46ef6f2094949d8831f13,875e50d28d26088a760866c1125ff124
219471,875e50d28d26088a760866c1125ff124,5281fb229131fa372bd15589fed81bc7
219472,5281fb229131fa372bd15589fed81bc7,ac398d9f0ba05d231b9d183feb595b12
219476,ac398d9f0ba05d231b9d183feb595b12,875e50d28d26088a760866c1125ff124
410473,ac398d9f0ba05d231b9d183feb595b12,38295c4cb820734391afed2c6eab0a22
